In [ ]:
from model import TwoLayerConv
from dataset import CSVLoader, UnNormalize
from train import train_model
from eval import evaluate
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch

## User model, loss function, optimizer, and hyper-parameters

In [ ]:
# Hyper-parameters
epochs     = 50
lr         = 0.001
batch_size = 32

# Model, Loss_fn, and Optimizer
model      = TwoLayerConv()
criterion  = torch.nn.CrossEntropyLoss()
optimizer  = torch.optim.Adam(params = model.parameters(), lr = lr)
device     = torch.device('cuda:3')

## Load the data and create dataloaders

In [ ]:
cifar_mu = [0.49139968, 0.48215827, 0.44653124]
cifar_sd = [0.24703233, 0.24348505, 0.26158768]

tf = transforms.Compose([transforms.ToTensor(),
                         transforms.Normalize(mean = cifar_mu, std = cifar_sd),
                         transforms.RandomHorizontalFlip()
                         ])

tf_test = transforms.Compose([transforms.ToTensor(),
                         transforms.Normalize(mean = cifar_mu, std = cifar_sd)
                         ])


# Downloaded from pytorch
dataset = datasets.CIFAR10(root='data/', download = True, transform = tf)
test_dataset = datasets.CIFAR10(root='data/', train = False, transform = tf_test)

train_dataset, val_dataset = random_split(dataset, [45000, 5000])

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 4)
val_loader   = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 4)
test_loader  = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, num_workers = 4)

## Example Images

In [ ]:
im_tr = transforms.Compose([UnNormalize(cifar_mu, cifar_sd),
                            transforms.ToPILImage()])

images = next(iter(train_loader))[0]
_, ax = plt.subplots(3, 3)

counter = 0
for i in range(3):
    for j in range(3):
        
        image = im_tr(images[counter])
        ax[i, j].imshow(image)
        counter += 1

## Tranining Begins!

In [ ]:
model, losses, accuracies, val_losses, val_accuracies = train_model(model, epochs, train_loader,
 val_loader, optimizer, criterion, device, track_loss = True)

## Evalute Model on Test Set

In [ ]:
loss, accuracy = evaluate(model, test_loader, criterion, device)
accuracy * 100

## Plot Accuracies and Losses

In [ ]:
plt.plot(accuracies, label = 'Train')
plt.legend()
plt.plot(val_accuracies, label = 'Val')
plt.legend()
plt.title('Accuracies')

In [ ]:
plt.plot([loss * .10 for loss in losses], label = 'Train')
plt.legend()
plt.plot([val_loss * .90 for val_loss in val_losses], label = 'Val')
plt.legend()
plt.title('Losses')